In [3]:
import pandas as pd
import numpy as np

In [21]:
#calculate total weighted tardiness of job_list

def weightt(jobs_list):
    wt = 0
    for i in range(20):
        sump = 0
        for j in range(i+1):
            sump += jobs_list[j][1]
        w = jobs_list[i][3]
        d = jobs_list[i][2]
        t = max(sump-d,0)
        wt += w*t
    return wt

In [22]:
#count tardy jobs in job_list

def tardyjobs(jobs_list):
    tj = 0
    for i in range(20):
        sump = 0
        for j in range(i+1):
            sump += jobs_list[j][1]
        d = jobs_list[i][2]
        
        if sump-d > 0:
            tj += 1
    return tj

In [23]:
# compare weighted tardinessall of all inititial solutions 
# to get the best job list (with min weighted tardiness)
# meanwhile, update tabulist and next job list

def jobswap(jobs, tabulist, tabelenth):
    min = 9999999
    tabu = []
    for i in range(19):
        #print(jobs)
        jobs_temp = []
        for j in range(20):
            jobs_temp.append(jobs[j])
        jobs_temp[i], jobs_temp[i+1] = jobs_temp[i+1], jobs_temp[i]
        #print(jobs_temp)
        
        #print([jobs_temp[i], jobs_temp[i+1]])
        if [jobs_temp[i], jobs_temp[i+1]] in tabulist:
            #print('tabu!')
            continue
        
        else:
            wt = weightt(jobs_temp)
            if wt<min:
                min = wt
                tabu = [jobs[i],jobs[i+1]]
                jobs_next = jobs_temp
                
    tabulist.append(tabu)
    if len(tabulist)>tabulenth:
        tabulist.pop(0)

    return min,tabu,jobs_next,tabulist

In [36]:
#do tabu search umtil fits stop_criterion(min_best(min weighted tardiness) not updated for (stop_criterion) times)

def tabusearch(stop_criterion, tabulenth):
    # initialize jobs_best, min_best(min weighted tardiness)
    tabulist=[]
    jobs =[(1,10, 50, 10), (2,10, 38, 5), (3, 13, 49, 1), (4, 4, 12, 5), (5, 9, 20, 10), (6, 4, 105, 1), (7, 8, 73, 5), (8, 15, 45, 10), 
               (9, 7, 6, 5), (10, 1, 64, 1), (11, 9, 15, 5), (12, 3, 6, 10), (13, 15, 92, 10), (14, 9, 43, 5), (15,11, 78, 1), 
               (16, 6, 21, 10), (17, 5, 15, 5), (18, 14, 50, 5), 
               (19, 18, 150, 1), (20, 3, 99, 5)]

    jobs_best = [(1,10, 50, 10), (2,10, 38, 5), (3, 13, 49, 1), (4, 4, 12, 5), (5, 9, 20, 10), (6, 4, 105, 1), (7, 8, 73, 5), (8, 15, 45, 10), 
               (9, 7, 6, 5), (10, 1, 64, 1), (11, 9, 15, 5), (12, 3, 6, 10), (13, 15, 92, 10), (14, 9, 43, 5), (15,11, 78, 1), 
               (16, 6, 21, 10), (17, 5, 15, 5), (18, 14, 50, 5), 
               (19, 18, 150, 1), (20, 3, 99, 5)]

    #terminate criterion: min_best(min weighted tardiness) not updated for (stop_criterion) times
    min_best = 9999999
    stop = stop_criterion

    while(stop):
        
        min,tabu,jobs_next,tabulist = jobswap(jobs,tabulist,tabulenth)
        
        # if achieve lower weighted tardiness, update the best solution
        if min <min_best:
            min_best = min
            jobs_best = jobs_next
            stop = stop_criterion
        else:
            stop -= 1
            
        jobs = jobs_next
    return min_best, jobs_best

In [61]:
import time

#experienments fo each stopcriterion, tabulenth

stopcriterions = [3,5,10,20]
tabulenths = [1,2,3,4]
experiments = []

for s in stopcriterions:
    for t in tabulenths:
        start_time = time.time()
        min_best, jobs_best = tabusearch(s, t)

        job_seq = ""
        for i in range(20):
            job_seq += str(jobs_best[i][0])
            job_seq += ','
            
        tardy_jobs = tardyjobs(jobs_best)
        
        experiments.append([s, t, min_best, job_seq, tardy_jobs, (time.time() - start_time)])

In [62]:
df = pd.DataFrame(experiments, columns = ['stop criterion', 'tabulist length', 'min weighted tardiness',\
                                          'optimal job seq.', '# of tardy jobs', 'execution time'])

In [63]:
df.style.set_properties(subset=['optimal job seq.'], **{'width': '300px'})

,stop criterion,tabulist length,min weighted tardiness,optimal job seq.,# of tardy jobs,execution time
0,3,1,2300,"12,9,4,16,5,17,2,1,8,10,7,3,20,13,11,14,18,6,15,19,",17,0.119678
1,3,2,2300,"12,9,4,16,5,17,2,1,8,10,7,3,20,13,11,14,18,6,15,19,",17,0.062052
2,3,3,2300,"12,9,4,16,5,17,2,1,8,10,7,3,20,13,11,14,18,6,15,19,",17,0.046864
3,3,4,2300,"12,9,4,16,5,17,2,1,8,10,7,3,20,13,11,14,18,6,15,19,",17,0.064257
4,5,1,1984,"12,9,4,16,5,17,2,1,8,10,7,13,20,14,11,18,6,15,3,19,",17,0.042991
5,5,2,1984,"12,9,4,16,5,17,2,1,8,10,7,13,20,14,11,18,6,15,3,19,",17,0.062551
6,5,3,1984,"12,9,4,16,5,17,2,1,8,10,7,13,20,14,11,18,6,15,3,19,",17,0.046863
7,5,4,1984,"12,9,4,16,5,17,2,1,8,10,7,13,20,14,11,18,6,15,3,19,",17,0.062462
8,10,1,1984,"12,9,4,16,5,17,2,1,8,10,7,13,20,14,11,18,6,15,3,19,",17,0.093689
9,10,2,1984,"12,9,4,16,5,17,2,1,8,10,7,13,20,14,11,18,6,15,3,19,",17,0.093767
